In [2]:
import itertools
import numpy as np
import pandas as pd
import scipy.stats as scs
from scipy.optimize import curve_fit
from numpy import linalg
from localcider.sequenceParameters import SequenceParameters
from Bio import SeqIO

In [3]:
# Parameter from calvadoss
residues = pd.read_csv('residues.csv')
residues = residues.set_index('one')

In [4]:
def calc_seq_prop(seq,residues,Nc=1,Cc=1,Hc=0):
    """df: DataFrame to be populated with sequence properties
    r: DataFrame of aa-specific parameters"""
    seq = list(seq).copy()   
    fasta_kappa = np.array(seq.copy())
    N = len(seq)
    r = residues.copy()
    
    # calculate properties that do not depend on charges
    fK = sum([seq.count(a) for a in ['K']])/N
    fR = sum([seq.count(a) for a in ['R']])/N
    fE = sum([seq.count(a) for a in ['E']])/N
    fD = sum([seq.count(a) for a in ['D']])/N
    faro = sum([seq.count(a) for a in ['W','Y','F']])/N
    mean_lambda = np.mean(r.loc[seq].lambdas)

    pairs = np.array(list(itertools.combinations(seq,2)))
    pairs_indices = np.array(list(itertools.combinations(range(N),2)))
    # calculate sequence separations
    ij_dist = np.diff(pairs_indices,axis=1).flatten().astype(float)
    # calculate lambda sums
    ll = r.lambdas.loc[pairs[:,0]].values+r.lambdas.loc[pairs[:,1]].values
    # calculate SHD
    beta = -1
    shd = np.sum(ll*np.power(np.abs(ij_dist),beta))/N
    SeqOb = SequenceParameters(''.join(seq))
    omega = SeqOb.get_kappa_X(grp1=['F','Y','W'])
    
    # fix charges
    if Nc == 1:
        r.loc['X'] = r.loc[seq[0]]
        r.loc['X','q'] = r.loc[seq[0],'q'] + 1.
        seq[0] = 'X'
        if r.loc['X','q'] > 0:
            fasta_kappa[0] = 'K'
        else:
            fasta_kappa[0] = 'A'
    if Cc == 1:
        r.loc['Z'] = r.loc[seq[-1]]
        r.loc['Z','q'] = r.loc[seq[-1],'q'] - 1.
        seq[-1] = 'Z'
        if r.loc['Z','q'] < 0:
            fasta_kappa[-1] = 'D'
        else:
            fasta_kappa[-1] = 'A'
    if Hc < 0.5:
        r.loc['H', 'q'] = 0
        fasta_kappa[np.where(np.array(seq) == 'H')[0]] = 'A'
    elif Hc >= 0.5:
        r.loc['H', 'q'] = 1
        fasta_kappa[np.where(np.array(seq) == 'H')[0]] = 'K'
            
    # calculate properties that depend on charges
    pairs = np.array(list(itertools.combinations(seq,2)))
    # calculate charge products
    qq = r.q.loc[pairs[:,0]].values*r.q.loc[pairs[:,1]].values
    # calculate SCD
    scd = np.sum(qq*np.sqrt(ij_dist))/N
    SeqOb = SequenceParameters(''.join(fasta_kappa))
    kappa = SeqOb.get_kappa()
    fcr = r.q.loc[seq].abs().mean()
    ncpr = r.q.loc[seq].mean()

    return np.around([fK, fR, fE, fD, faro, mean_lambda, shd, omega, scd, kappa, fcr, ncpr],3)

In [6]:
df = pd.DataFrame(columns=['fK','fR','fE','fD','fARO','Mean_lambda','SHD','Omega_ARO','SCD','kappa','FCR','NCPR'])

for record in SeqIO.parse("ADan_2_df.fasta", "fasta"): 
    df.loc[record.id] = calc_seq_prop(record.seq, residues, Nc=1, Cc=1, Hc=0)

df.index.name = "ID"
df

,fK,fR,fE,fD,fARO,Mean_lambda,SHD,Omega_ARO,SCD,kappa,FCR,NCPR
ID,,,,,,,,,,,,
E-1-K,0.088,0.029,0.088,0.0,0.176,0.473,2.956,0.272,-0.537,0.073,0.265,0.029
E-1-N,0.059,0.029,0.088,0.0,0.176,0.480,2.985,0.272,-0.341,0.073,0.235,0.000
E-1-T,0.059,0.029,0.088,0.0,0.176,0.479,2.979,0.272,-0.341,0.073,0.235,0.000
E-1-R,0.059,0.059,0.088,0.0,0.176,0.489,3.022,0.272,-0.537,0.073,0.265,0.029
E-1-S,0.059,0.029,0.088,0.0,0.176,0.481,2.990,0.272,-0.341,0.073,0.235,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...
E-1-V,0.059,0.029,0.088,0.0,0.176,0.474,2.959,0.272,-0.341,0.073,0.235,0.000
E-1-Y,0.059,0.029,0.088,0.0,0.206,0.496,3.052,0.234,-0.341,0.073,0.235,0.000
E-1-W,0.059,0.029,0.088,0.0,0.206,0.497,3.053,0.234,-0.341,0.073,0.235,0.000


In [7]:
df.to_csv("ADan_2_df_localCider_calvadoss.csv")

Run Albatros at: https://colab.research.google.com/github/holehouse-lab/ALBATROSS-colab/blob/main/example_notebooks/polymer_property_predictors.ipynb